# Material Complementar da Aula 15

In [ ]:
import networkx as nx
import freeman as fm

from math import isclose
from random import choice, shuffle

Os resultados do notebook anterior foram originalmente obtidos pelos pesquisadores Vincent Buskens e Arnout van de Rijt no artigo *Dynamics of networks if everyone strives for structural holes*. Os resultados implicam que, se todo mundo tenta levar vantagem, ninguém na prática leva muita vantagem no final.

No entanto, cabe observar que "everyone strives for structural holes" é uma premissa bem forte! Nem todo mundo é um "striver" por lacunas estruturais, talvez alguns empreendedores fiquem na deles ou usem estratégias diferentes.

Vamos então repensar a simulação. Comecemos pela parte do código que permanece igual, salvo o nome da função `constraint` que foi substituído pela função `striver_constraint`.

In [ ]:
def investment(g, n, m):
    if g.has_edge(n, m):
        return 1 / g.degree(n)
    return 0

def local_constraint(g, n, m):
    i = investment(g, n, m)
    for k in g.neighbors(n):
        i += investment(g, n, k) * investment(g, k, m)
    return i**2

def striver_constraint(g, n):
    if g.degree(n) == 0:
        return 2
    c = 0
    for m in g.neighbors(n):
        c += local_constraint(g, n, m)
    return c

É sempre uma boa ideia evitar muita variedade de parâmetros logo de cara, por causa da potencial explosão combinatória de possibilidades para simular. Assim, vamos propor uma extensão que divide agentes em dois tipos: os "strivers" originais e os "hipsters" que têm algum tipo diferente de comportamento. Disso segue uma nova função `constraint`.

In [ ]:
def constraint(g, n):
    if g.nodes[n]['agent'] == 'striver':
        return striver_constraint(g, n)
    return hipster_constraint(g, n)

Temos então duas variáveis a considerar:

1. qual exatamente é o comportamento desses "hipsters";

2. qual é a proporção de "strivers" e "hipsters".

O segundo pode ser facilmente transformado em um novo parâmetro da simulação. Vamos propor uma nova subclasse de `Simulation`.

In [ ]:
class HoleSimulation(fm.Simulation):


    # Agora, além de receber o número de nós, o construtor
    # também recebe o número de strivers. Por simplicidade,
    # entende-se que todos os outros agentes são hipsters.
    def __init__(self, num_nodes, num_strivers):
        self.num_nodes = num_nodes
        self.num_strivers = num_strivers


    # Agora, além de construir o grafo, escolhemos de forma
    # aleatória quais nós são strivers e quais são hipsters.
    def before_each(self):
        g = fm.Graph(nx.erdos_renyi_graph(self.num_nodes, 0.5))
        nodes = list(g.nodes)
        shuffle(nodes)
        for n in nodes[:self.num_strivers]:
            g.nodes[n]['agent'] = 'striver'
        for n in nodes[self.num_strivers:]:
            g.nodes[n]['agent'] = 'hipster'
        self.g = g


    # Já a iteração permanece exatamente a mesma de antes.
    def iterate(self):
        g = self.g
        constraints = {}
        nodes = list(g.nodes)
        shuffle(nodes)
        for n in nodes:
            if n not in constraints:
                constraints[n] = constraint(g, n)
            impacts = {}
            for m in nodes:
                if n != m:
                    if m not in constraints:
                        constraints[m] = constraint(g, m)
                    g.flip_existence(n, m)
                    if g.has_edge(n, m) and constraint(g, m) > constraints[m]:
                        impacts[m] = 0
                    else:
                        impacts[m] = constraint(g, n) - constraints[n]
                    g.flip_existence(n, m)
            lowest = min(value for value in impacts.values())
            if lowest < 0:
                m = choice([m for m in impacts if isclose(impacts[m], lowest)])
                g.flip_existence(n, m)
                return True
        return False


    # Aqui não mudamos quase nada. Mas em vez de adicionar
    # os ids dos nós, adicionamos o perfil de cada um.
    def after_each(self, repetition, iterations, elapsed):
        g = self.g

        b = nx.betweenness_centrality(g)

        for n in g.nodes:
            self.append({
                'agent': g.nodes[n]['agent'],
                'betweenness': b[n],
            })

        # Como o conjunto de simulações é demorado, vamos
        # usar o método print para ficar dando feedbacks.
        # Temos à disposição, como parâmetros deste método,
        # algumas informações interessantes como o número
        # de iterações e o tempo consumido pela simulação.
        self.print({
            'num_strivers': self.num_strivers,
            'repetition': repetition,
            'iterations': iterations,
            'elapsed': elapsed,
        })

O primeiro resume-se a definir a função `hipster_constraint`. Podemos, por exemplo, definir que eles na verdade são totalmente indiferentes. Ou seja, a função sempre devolve zero independentemente da situação da rede.

In [ ]:
def hipster_constraint(g, n):
    return 0

Vamos então realizar várias simulações e visualizar os resultados.

In [ ]:
# Para maior segurança estatística, esses números
# deveriam ser maiores. Porém, isso deixaria a
# elaboração dos exercícios muito demorada.
NUM_NODES = 10
NUM_TIMES = 10


# Vamos guardar todos os resultados em um dicionário
# de dataframes. Chaves são números de strivers e
# valores são os respectivos dicionários obtidos.
dfs = {}

# Vamos variar o número de strivers de zero até o
# número total de nós (ou seja, zero hipsters).
for num_strivers in range(0, NUM_NODES + 1):
    s = HoleSimulation(NUM_NODES, num_strivers)

    # Como há fatores aleatórios envolvidos (ex:
    # construção da rede, distribuição dos agentes),
    # não podemos confiar em uma única simulação.
    # Então fazemos várias para considerar a média.
    # O método run da simulação aceita um parâmetro
    # times que permite especificar quantas repetições.
    dfs[num_strivers] = s.run(times=NUM_TIMES)

# A função concat junta todos os dataframes em um só,
# usando as chaves para distinguir as linhas de cada
# um. O segundo parâmetro é o nome de uma coluna
# adicional onde você quer guardar essas chaves.
df = fm.concat(dfs, 'number of strivers')

Vamos usar o método `linplot` para ver o betweenness médio (no final da simulação) em função do número de strivers. O parâmetro `control` serve para separar os gráficos de strivers e hipsters.

In [ ]:
fm.linplot(df, 'number of strivers', 'betweenness', control='agent')

Nesse exemplo, os resultados sugerem que, a partir de dois strivers, a situação degenera para o mesmo resultado original. No entanto, quando o striver é único, ele parece de fato conseguir um betweenness superior. Idealmente, seria bom fixar esse parâmetro e validar essa superioridade via teste-t.

## Exercício 1

Implemente uma versão da `hipster_constraint` que seja o completo oposto do "striver", ou seja, o comportamento de **rejeitar degree e querer redundância**. Para fins de comparação, o valor devolvido **não pode ser negativo**. Para evitar nós isolados, mantenha a restrição de que é excepcionalmente 2 quando o degree é zero.

**Dica:** exceto no caso em que o degree é zero, o valor máximo possível da `striver_constraint` é 9/8.

In [ ]:
def hipster_constraint(g, n):
    pass # seu código aqui

Interprete sucintamente os resultados e indique o que você poderia fazer para complementá-los.

*(seu texto aqui)*

## Exercício 2

Implemente uma versão da `hipster_constraint` que seja uma "versão parcial" do "striver", ou seja, o comportamento de **apenas querer degree**. Para fins de comparação, o valor devolvido **não pode ser negativo**. Não precisa se basear na fórmula original, há meios mais fáceis.

In [ ]:
def hipster_constraint(g, n):
    pass # seu código aqui

Interprete sucintamente os resultados e indique o que você poderia fazer para complementá-los.

*(seu texto aqui)*

## Exercício 3

Implemente uma versão da `hipster_constraint` que seja um "oposto parcial" do "striver", ou seja, o comportamento de **querer redundância**. Para fins de comparação, o valor devolvido **não pode ser negativo**. Não precisa se basear na fórmula original, há meios mais fáceis.

In [ ]:
def hipster_constraint(g, n):
    pass # seu código aqui

Interprete sucintamente os resultados e indique o que você poderia fazer para complementá-los.

*(seu texto aqui)*

E você ainda pode voltar ao handout para os últimos desafios...